### Set up django env

In [5]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'intelliwiz_config.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

### Import required modules

In [6]:
from apps.activity.models import Asset, Attachment, Device, DeviceEventlog,  Event, Job, Jobneed, JobneedDetails, Question, QuestionSet, QuestionSetBelonging,  WorkPermit
from apps.attendance.models import PeopleEventlog, TestGeo, Tracking
from apps.y_helpdesk.models import Ticket, EscalationMatrix
from apps.onboarding.models import Bt, Contract, ContractDetail, GeofenceMaster, Shift, SitePeople, TypeAssist, WizardDraft
from apps.peoples.models import Capability, People, PermissionGroup, Pgbelonging, Pgroup
from apps.tenants.models import Tenant
from django.contrib.admin.models import LogEntry
from django.contrib.auth.models import Group, Permission
from django.contrib.contenttypes.models import ContentType
from django.contrib.sessions.models import Session
# Shell Plus Django Imports
from django.core.cache import cache
from django.conf import settings
from django.contrib.auth import get_user_model
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When, ExpressionWrapper

from django.urls import reverse
from django.db.models import Exists, OuterRef, Subquery, Value
from django.db.models.functions import  ACos, Round, Cos, Radians, Concat
from django.contrib.gis.db.models.functions import  Cast
from django.contrib.gis.db.models.functions import Distance, AsGeoJSON
from django.db import models
import requests

In [6]:
from datetime import datetime, timedelta, timezone

In [5]:
import rt.rest2

api_url = 'https://rt.youtility.in/REST/2.0/'
username = 'root'
password = 'password'
c = rt.rest2.Rt(url=api_url, http_auth=requests.auth.HTTPBasicAuth(username, password))

In [5]:
c.user_exists('spsadmin3', privileged=False)



True

In [17]:
People.objects.filter(
peoplecode='YS0040'
).values('bu_id', 'designation__tacode', 'isverified')

<QuerySet [{'bu_id': 9, 'designation__tacode': 'SITEINCHARGE', 'isverified': True}]>

In [23]:
Jobneed.objects.filter(
            #Q(Q(parent_id__in = [1, -1]) | Q(parent_id__isnull=True)),
            alerts=True,
            identifier='INTERNALTOUR'
        ).values('plandatetime', 'expirydatetime', 'parent_id')

<QuerySet [{'plandatetime': datetime.datetime(2023, 1, 14, 4, 40, tzinfo=<UTC>), 'expirydatetime': datetime.datetime(2023, 1, 14, 4, 50, tzinfo=<UTC>), 'parent_id': 4584}, {'plandatetime': datetime.datetime(2023, 1, 14, 5, 50, tzinfo=<UTC>), 'expirydatetime': datetime.datetime(2023, 1, 14, 6, 0, tzinfo=<UTC>), 'parent_id': 4592}, {'plandatetime': datetime.datetime(2023, 1, 14, 11, 0, tzinfo=<UTC>), 'expirydatetime': datetime.datetime(2023, 1, 14, 11, 10, tzinfo=<UTC>), 'parent_id': 4738}, {'plandatetime': datetime.datetime(2023, 1, 14, 12, 20, tzinfo=<UTC>), 'expirydatetime': datetime.datetime(2023, 1, 14, 12, 30, tzinfo=<UTC>), 'parent_id': 6660}]>

In [16]:
from apps.service.utils import get_email_recipients, add_attachments, alert_observation
alert_observation(jn, True)

 2023-01-25 18:03:32,472  INFO        from method: alert_observation                 << Sending alert mail with subject [READINGS ALERT] Site with Name: ICICI SPS HOUSE having checklist [GATE CHECKLIST] - readings out of range >>
 2023-01-25 18:03:32,474  INFO        from method: alert_observation                 << Attachments are going to attach >>
 2023-01-25 18:03:32,487  INFO        from method: add_attachments                   << attachments are attached.... >>
 2023-01-25 18:03:32,489  INFO        from method: add_attachments                   << attachments are attached.... >>
 2023-01-25 18:03:32,490  INFO        from method: add_attachments                   << attachments are attached.... >>
 2023-01-25 18:03:32,492  INFO        from method: add_attachments                   << attachments are attached.... >>
 2023-01-25 18:03:38,687  INFO        from method: alert_observation                 << Alert mail sent to ['naveen.sargam@youtility.in', 'vaishvika.pawar@youtility.in

In [3]:
Jobneed.objects.filter(job_id=1055).values('plandatetime','expirydatetime').order_by('plandatetime', 'expirydatetime').distinct('plandatetime', 'expirydatetime')
Job.objects.filter(id=1055).values('fromdate', 'uptodate')

<QuerySet [{'fromdate': datetime.datetime(2023, 2, 1, 18, 30, tzinfo=<UTC>), 'uptodate': datetime.datetime(2023, 2, 4, 18, 29, tzinfo=<UTC>)}]>

In [6]:
from apps.schedhuler.utils import create_ppm_job
create_ppm_job(1068)

 2023-02-08 21:21:17,938  INFO        from method: create_ppm_job                    << processing jobs started found:= '1' jobs >>
 2023-02-08 21:21:17,940  INFO        from method: calculate_startdtz_enddtz_for_ppm  << calculating startdtz, enddtz for job:= [1068] >>
 2023-02-08 21:21:17,941  INFO        from method: del_job                           << deleting old jobs start[+] >>
 2023-02-08 21:21:17,947  INFO        from method: del_job                           << total jobneedetails deleted: 8 >>
 2023-02-08 21:21:17,949  INFO        from method: del_job                           << total jobneed deleted: 2 >>
 2023-02-08 21:21:17,951  INFO        from method: del_job                           << deleting old jobs end[-] >>
 2023-02-08 21:21:17,952  INFO        from method: del_ppm_reminder                  << del_ppm_reminder start + >>


/home/redmine/envs/youtility4/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Reminder.reminderdate received a naive datetime (2023-02-08 21:21:17.953416) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


 2023-02-08 21:21:17,955  INFO        from method: del_ppm_reminder                  << del_ppm_reminder end - >>
 2023-02-08 21:21:17,956  DEBUG       from method: create_ppm_job                    << Jobs to be schedhuled from startdatetime 2023-02-08 21:21:17+05:30 to enddatetime 2023-02-10 23:00:00+05:30 >>
 2023-02-08 21:21:17,957  INFO        from method: get_datetime_list                 << get_datetime_list(cron_exp, startdtz, enddtz) [start] >>
 2023-02-08 21:21:17,958  INFO        from method: get_datetime_list                 << getting datetime list for cron:=30 21 * * *, starttime:= '2023-02-08 21:21:17+05:30' and endtime:= '2023-02-10 23:00:00+05:30' >>
 2023-02-08 21:21:17,961  INFO        from method: get_datetime_list                 << Datetime list calculated are as follows:= [datetime.datetime(2023, 2, 8, 21, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=19800))),
 datetime.datetime(2023, 2, 9, 21, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=19

ic| 'resp in get_datetime_list()', resp: None


 2023-02-08 21:21:18,203  DEBUG       from method: create_ppm_job                    << Jobneed will going to create for all this datetimes
 ['08-Feb-2023 21:30', '09-Feb-2023 21:30', '10-Feb-2023 21:30'] >>
 2023-02-08 21:21:18,204  INFO        from method: insert_into_jn_and_jnd            << insert_into_jn_and_jnd() [ start ] >>
 2023-02-08 21:21:18,214  INFO        from method: to_utc                            << to_utc() start [+] >>
 2023-02-08 21:21:18,216  INFO        from method: to_utc                            << found total 3 datetimes >>
 2023-02-08 21:21:18,217  INFO        from method: to_utc                            << before conversion datetimes [datetime.datetime(2023, 2, 8, 21, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=19800))), datetime.datetime(2023, 2, 9, 21, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=19800))), datetime.datetime(2023, 2, 10, 21, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=19800)))] >>
 2023-02-08 21:21:18

...

 2023-02-08 21:21:18,268  INFO        from method: insert_update_jobneeddetails      << Checklist found with 4 questions in it. >>
 2023-02-08 21:21:18,269  INFO        from method: insert_into_jnd                   << insert_into_jnd() [START] >>
 2023-02-08 21:21:18,273  INFO        from method: insert_into_jnd                   << insert_into_jnd() [END] >>
 2023-02-08 21:21:18,274  INFO        from method: insert_update_jobneeddetails      << insert_update_jobneeddetails() [END] >>
 2023-02-08 21:21:18,275  INFO        from method: insert_into_jn_and_jnd            << createJob() parent jobneed:= 7524 >>
 2023-02-08 21:21:18,276  DEBUG       from method: insert_into_jn_and_jnd            << pdtz:=2023-02-10 16:00:00+00:00 edtz:=2023-02-10 16:15:00+00:00 >>
 2023-02-08 21:21:18,278  INFO        from method: insert_update_jobneeddetails      << insert_update_jobneeddetails() [START] >>


ic| parent: False, job['qset_id']: 3


 2023-02-08 21:21:18,291  INFO        from method: insert_update_jobneeddetails      << Checklist found with 4 questions in it. >>
 2023-02-08 21:21:18,292  INFO        from method: insert_into_jnd                   << insert_into_jnd() [START] >>
 2023-02-08 21:21:18,296  INFO        from method: insert_into_jnd                   << insert_into_jnd() [END] >>
 2023-02-08 21:21:18,297  INFO        from method: insert_update_jobneeddetails      << insert_update_jobneeddetails() [END] >>
 2023-02-08 21:21:18,298  INFO        from method: insert_into_jn_and_jnd            << createJob() parent jobneed:= 7525 >>
 2023-02-08 21:21:18,299  INFO        from method: update_lastgeneratedon            << update_lastgeneratedon [start] >>
 2023-02-08 21:21:18,300  INFO        from method: update_lastgeneratedon            << after lastgenreatedon:=2023-02-10 16:00:00+00:00 >>
 2023-02-08 21:21:18,302  INFO        from method: update_lastgeneratedon            << update_lastgeneratedon [end] >>
 2

/home/redmine/envs/youtility4/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Jobneed.plandatetime received a naive datetime (2023-02-08 21:21:18.304417) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


 2023-02-08 21:21:18,317  INFO        from method: create_ppm_reminder               << create_ppm_reminder() jobneed:7523 plandatetime 2023-02-08 16:00:00+00:00 jobdesc TestPPMFeb82022 buid 9 asset_id 50 qsetid 3 >>
 2023-02-08 21:21:18,319  DEBUG       from method: create_ppm_reminder               << reminder : <EscalationMatrix: EscalationMatrix object (3)> >>
 2023-02-08 21:21:18,321  INFO        from method: create_ppm_reminder               << create_ppm_reminder() jobneed:7524 plandatetime 2023-02-09 16:00:00+00:00 jobdesc TestPPMFeb82022 buid 9 asset_id 50 qsetid 3 >>
 2023-02-08 21:21:18,322  DEBUG       from method: create_ppm_reminder               << reminder : <EscalationMatrix: EscalationMatrix object (3)> >>
 2023-02-08 21:21:18,324  INFO        from method: create_ppm_reminder               << create_ppm_reminder() jobneed:7525 plandatetime 2023-02-10 16:00:00+00:00 jobdesc TestPPMFeb82022 buid 9 asset_id 50 qsetid 3 >>
 2023-02-08 21:21:18,325  DEBUG       from method

## Request Tracker R&D

In [7]:
Ticket.objects.values()

<QuerySet [{'tenant_id': None, 'cuser_id': 8, 'muser_id': 8, 'cdtz': datetime.datetime(2023, 2, 3, 13, 1, 31, tzinfo=<UTC>), 'mdtz': datetime.datetime(2023, 2, 3, 13, 3, 12, tzinfo=<UTC>), 'ctzoffset': -1, 'uuid': UUID('4a12f705-279b-41f8-ae5d-c46486a41789'), 'id': 1, 'ticketdesc': 'test', 'assignedtopeople_id': 11, 'assignedtogroup_id': 1, 'comments': 'fdsafsaf', 'bu_id': 9, 'client_id': None, 'priority': None, 'ticketcategory_id': 1, 'location_id': 5, 'modifieddatetime': datetime.datetime(2023, 2, 3, 13, 3, 12, 348235, tzinfo=<UTC>), 'level': 0, 'status': None, 'performedby_id': None, 'ticketlog': {'statusjbdata': []}, 'events': None, 'isescalated': False, 'ticketsource': None}, {'tenant_id': None, 'cuser_id': 8, 'muser_id': 8, 'cdtz': datetime.datetime(2023, 2, 3, 13, 19, 26, tzinfo=<UTC>), 'mdtz': datetime.datetime(2023, 2, 3, 13, 19, 46, tzinfo=<UTC>), 'ctzoffset': -1, 'uuid': UUID('aca41daf-529d-42f4-bfd6-d753bb82f579'), 'id': 2, 'ticketdesc': 'testDescription', 'assignedtopeople

In [10]:
t = TypeAssist.objects.filter(id=206).first()

In [11]:
t.esc_types.select_related('escalationtemplate', 'assignedperson', 'assignedgroup').all()

<QuerySet [<EscalationMatrix: EscalationMatrix object (1)>]>

In [12]:
T = TypeAssist.objects.filter(tacode="TESTINGENABLE").order_by('-cdtz').first()

In [13]:
T.enable

True

In [4]:
j  = Job.objects.get(id=1068)

In [6]:
from apps.core import utils

In [4]:
now = datetime.now()

In [7]:
utils.to_utc([now])

 2023-02-08 15:53:02,013  INFO        from method: to_utc                            << to_utc() start [+] >>
 2023-02-08 15:53:02,016  INFO        from method: to_utc                            << found total [datetime.datetime(2023, 2, 8, 15, 52, 55, 750495)] datetimes >>
 2023-02-08 15:53:02,017  INFO        from method: to_utc                            << datetime list returned dtlist=[datetime.datetime(2023, 2, 8, 10, 22, 55, tzinfo=<UTC>)] >>


[datetime.datetime(2023, 2, 8, 10, 22, 55, tzinfo=<UTC>)]

In [3]:
from apps.service.tasks import autoclose_job, send_reminder_email


 2023-02-10 08:38:32,264  WARNING     from method: ModelSchemaField                  << <class 'django.contrib.gis.db.models.fields.PointField'> is currently unhandled, defaulting to str. >>


In [4]:
send_reminder_email()

TypeError: unsupported type for timedelta minutes component: F

In [13]:
Jobneed.objects.filter(id=7340).update(
other_info['email_sent'] = True)

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (1106846057.py, line 2)